In [3]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import pandas as pd

In [6]:
base_url = "https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page={}"

# Number of pages you want to scrape
num_pages = 252

# Create a list of base URLs with page numbers
base_urls = [base_url.format(page) for page in range(0, num_pages + 1)]


In [7]:
# Create a CSV file to write the data
with open('etsy_products.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row to the CSV file
    csv_writer.writerow(['Title', 'Price', 'URL'])

    for url in base_urls:
        print("Scraping URL:", url)  # Add this line for debugging
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to retrieve the page {url}. Status code:", response.status_code)
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all product elements on the page
        product_elements = soup.find_all('a', class_='listing-link')

        # Extract and write the data to the CSV file
        for product in product_elements:
            title_element = product.find('h3', class_='wt-text-caption')
            title = title_element.get_text(strip=True) if title_element else "N/A"

            price_element = product.find('span', class_='currency-value')
            price = price_element.get_text(strip=True) if price_element else "N/A"

            product_url = product['href']

            csv_writer.writerow([title, price, product_url])

print("Data has been scraped and saved to 'etsy_products.csv'.")


Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=0
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=1
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=2
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=3
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=4
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=5
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=6
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=7
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=8
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=9
Scraping URL: https://www.etsy

In [8]:
df = pd.read_csv('/content/etsy_products.csv')
df

,Title,Price,URL
0,Classy Printed & Work White and Red Tunic Comb...,"4,303",https://www.etsy.com/in-en/listing/1565898204/...
1,Silk Sari Long Wrap Skirts,851,https://www.etsy.com/in-en/listing/1407803519/...
2,Designer ethnic lehenga choli for wedding func...,"2,749",https://www.etsy.com/in-en/listing/1483803728/...
3,"Premium Tussar Silk Patola Lehenga choli, chan...","3,023",https://www.etsy.com/in-en/listing/1449046514/...
4,Designer ethnic lehenga choli for wedding func...,"3,500",https://www.etsy.com/in-en/listing/1420022127/...
...,...,...,...
12280,Pencil skirt. Beautiful fitted black pencil sk...,"6,198",https://www.etsy.com/in-en/listing/1072772090/...
12281,90s Y2K Plaid Wrap Skirt/ US 6,"2,528",https://www.etsy.com/in-en/listing/1310137907/...
12282,Pink Suede Patchwork Mini Skirt,"6,198",https://www.etsy.com/in-en/listing/1576318817/...
12283,African Skirt/Plus Size Skirt/Skirt/African Cl...,"6,208",https://www.etsy.com/in-en/listing/1027438184/...


In [10]:
# Create a CSV file to write the data
csv_filename = 'etsy_products.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header row to the CSV file
    csv_writer.writerow(['Title', 'Price', 'URL', 'Original Price', 'Discount','Delivery Type','Stock'])

    for url in base_urls:
        print("Scraping URL:", url)  # Add this line for debugging
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to retrieve the page {url}. Status code:", response.status_code)
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all product elements on the page
        product_elements = soup.find_all('a', class_='listing-link')

        # Extract and write the data to the CSV file
        for product in product_elements:
            title_element = product.find('h3', class_='wt-text-caption')
            title = title_element.get_text(strip=True) if title_element else "N/A"

            price_element = product.find('span', class_='currency-value')
            price = price_element.get_text(strip=True) if price_element else "N/A"

            delivery_element = product.find('span', class_='wt-badge wt-badge--small wt-badge--statusValue')
            delivery = delivery_element.get_text(strip=True) if delivery_element else "Delivery charged"

            product_url = product['href']

            # You'll need to find the elements for Original Price (replace with actual class name)
            original_price_element = product.find('span', class_='wt-text-strikethrough')
            original_price = original_price_element.get_text(strip=True) if original_price_element else price  # Set original price to price if it's blank

            # Remove currency symbol and commas, and then convert to float
            if original_price != "N/A":
                original_price = original_price.replace('₹', '').replace(',', '').strip()  # Remove currency symbol and commas
                original_price = float(original_price)  # Convert to float

            # Calculate discount if both original price and price are available
            if original_price != "N/A" and price != "N/A":
                price = price.replace(',', '').strip()  # Remove commas
                price = float(price)  # Convert to float

                discount = original_price - price
            else:
                discount = "N/A"

          # Set the stock status based on the availability of the price
            stock = "In Stock" if price != "N/A" else "Out of Stock"

            csv_writer.writerow([title, price, product_url, original_price, discount, delivery, stock])

print("Data has been scraped and saved to 'etsy_products.csv'.")


Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=0
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=1
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=2
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=3
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=4
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=5
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=6
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=7
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=8
Scraping URL: https://www.etsy.com/in-en/c/clothing/womens-clothing/skirts?ref=pagination&page=9
Scraping URL: https://www.etsy

In [11]:
df = pd.read_csv('/content/etsy_products.csv')
df

,Title,Price,URL,Original Price,Discount,Delivery Type,Stock
0,Silk Sari Long Wrap Skirts,851.0,https://www.etsy.com/in-en/listing/1407803519/...,2128.0,1277.0,FREE delivery,In Stock
1,Classy Printed & Work White and Red Tunic Comb...,4303.0,https://www.etsy.com/in-en/listing/1565898204/...,4303.0,0.0,FREE delivery,In Stock
2,Designer ethnic lehenga choli for wedding func...,2749.0,https://www.etsy.com/in-en/listing/1483803728/...,5499.0,2750.0,FREE delivery,In Stock
3,Designer ethnic lehenga choli for wedding func...,3500.0,https://www.etsy.com/in-en/listing/1420022127/...,7000.0,3500.0,FREE delivery,In Stock
4,5 Pcs Lot Hippie Multi Colourful Cotton Handma...,1163.0,https://www.etsy.com/in-en/listing/1243981428/...,3879.0,2716.0,FREE delivery,In Stock
...,...,...,...,...,...,...,...
12280,Purple Tribal Pencil Skirt,1897.0,https://www.etsy.com/in-en/listing/723074492/p...,1897.0,0.0,Delivery charged,In Stock
12281,Vintage Wrap Skirt Requirements Large Blue Gre...,2715.0,https://www.etsy.com/in-en/listing/1159086483/...,3017.0,302.0,Delivery charged,In Stock
12282,Butterfly Wings rave set,1725.0,https://www.etsy.com/in-en/listing/1224548982/...,1725.0,0.0,Delivery charged,In Stock
12283,"1970s Finnish Arola wool skirt, 70s Hand loome...",8880.0,https://www.etsy.com/in-en/listing/1561738782/...,8880.0,0.0,Delivery charged,In Stock


Presently, there exist null (NaN) values that necessitate removal.

In [12]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/etsy_products.csv')

# Remove rows containing NaN values
df.dropna(inplace=True)

# Save the cleaned DataFrame back to a CSV file
df.to_csv('cleaned_file.csv', index=False)


In [13]:
df

,Title,Price,URL,Original Price,Discount,Delivery Type,Stock
0,Silk Sari Long Wrap Skirts,851.0,https://www.etsy.com/in-en/listing/1407803519/...,2128.0,1277.0,FREE delivery,In Stock
1,Classy Printed & Work White and Red Tunic Comb...,4303.0,https://www.etsy.com/in-en/listing/1565898204/...,4303.0,0.0,FREE delivery,In Stock
2,Designer ethnic lehenga choli for wedding func...,2749.0,https://www.etsy.com/in-en/listing/1483803728/...,5499.0,2750.0,FREE delivery,In Stock
3,Designer ethnic lehenga choli for wedding func...,3500.0,https://www.etsy.com/in-en/listing/1420022127/...,7000.0,3500.0,FREE delivery,In Stock
4,5 Pcs Lot Hippie Multi Colourful Cotton Handma...,1163.0,https://www.etsy.com/in-en/listing/1243981428/...,3879.0,2716.0,FREE delivery,In Stock
...,...,...,...,...,...,...,...
12279,KURON African Print Pencil Skirt | Ankara Midi...,6268.0,https://www.etsy.com/in-en/listing/1166967652/...,6268.0,0.0,Delivery charged,In Stock
12280,Purple Tribal Pencil Skirt,1897.0,https://www.etsy.com/in-en/listing/723074492/p...,1897.0,0.0,Delivery charged,In Stock
12281,Vintage Wrap Skirt Requirements Large Blue Gre...,2715.0,https://www.etsy.com/in-en/listing/1159086483/...,3017.0,302.0,Delivery charged,In Stock
12282,Butterfly Wings rave set,1725.0,https://www.etsy.com/in-en/listing/1224548982/...,1725.0,0.0,Delivery charged,In Stock
